## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_1 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o1/o1_bincore6.csv", index_col=0)
print(bin6_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_1.reset_index(inplace=True, drop=True)

print('shape -', bin6_1.shape)
print('reset_index 완료')
print('input data shape')
bin6_1.head()

(27602871, 2)
shape - (27602871, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_1['label'].value_counts())

256
0    27564156
1       38715
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_1[bin6_1['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 48 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_1), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_1', len(ls_idx_bin))

# loc 로 수정필요
bin6_1_Ngram = bin6_1.loc[ls_idx_bin,:].copy()

1858320
나머지 0
최종 길이 1858320
bin6_1 1858320


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_1_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_1.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_1[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

38715.0
0 23677488
1000 3284725
2000 10969134
3000 22384354
4000 3555044
5000 11494509
6000 6450245
7000 10932128
8000 24384270
9000 24199269
10000 17972757
11000 27410448
12000 21581974
13000 24890455
14000 25127906
15000 15921463
16000 4168073
17000 10099366
18000 7839429
19000 10271964
20000 12084921
21000 14301454
22000 14460902
23000 23600265
24000 2929470
25000 2614583
26000 11116030
27000 10912633
28000 22635850
29000 13162517
30000 9888318
31000 14154549
32000 8859887
33000 8505087
34000 18787183
35000 5456631
36000 930633
37000 25813049
38000 7091816
완료
38715


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_1['label'].value_counts()[1])

38715
38715


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_1_Ngram])
final.shape

(3716640, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_1_onehot_Ngram = pd.concat([final['label'], bc6_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_1_onehot_Ngram.shape)

원핫인코딩완료
(3716640, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_1 = bc6_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_1 = bc6_1_onehot_Ngram['label'].to_numpy()
print(x_bc6_1.shape, x_bc6_1.shape)

x_bc6_1 = x_bc6_1.reshape(-1, right_idx, x_bc6_1.shape[1])
y_bc6_1 = y_bc6_1.reshape(-1, right_idx, 1)

print(x_bc6_1.shape, y_bc6_1.shape)

(3716640, 256) (3716640, 256)
(77430, 48, 256) (77430, 48, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_1.shape[0])

x_bc6_1 = x_bc6_1[p]
y_bc6_1 = y_bc6_1[p]

print(x_bc6_1.shape, y_bc6_1.shape)

(77430, 48, 256) (77430, 48, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(72, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_1, y_bc6_1):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_1[train],
               y_bc6_1[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_1[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_1[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 227s 3ms/sample - loss: 0.0121 - accuracy: 0.9965
Epoch 2/10
69687/69687 [==============================] - 257s 4ms/sample - loss: 0.0020 - accuracy: 0.9994
Epoch 3/10
69687/69687 [==============================] - 252s 4ms/sample - loss: 0.0012 - accuracy: 0.9996
Epoch 4/10


accuracy_score 0.9998547074777218
recall_score 0.9956450734893849
precision_score 0.9945622620989668
f1_score 0.9951033732317738
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 320s 5ms/sample - loss: 1.9709e-04 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 167s 2ms/sample - loss: 1.4347e-04

accuracy_score 0.9999515691592405
recall_score 0.9987111029276377
precision_score 0.9979760809567617
f1_score 0.9983434566537824
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 308s 4ms/sample - loss: 7.0368e-05 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 249s 4ms/sample - loss: 5.5060e-05

accuracy_score 0.999986546988678
recall_score 0.9998169838945827
precision_score 0.9992683372965063
f1_score 0.9995425853078401
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 293s 4ms/sample - loss: 7.8270e-06 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 240s 3ms/sample - loss: 8.6282e-06 

accuracy_score 0.9999946187954711
recall_score 1.0
precision_score 0.9996438112199466
f1_score 0.9998218738867118
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 176s 3ms/sample - loss: 6.3908e-06 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 161s 2ms/sample - loss: 5.5971e-06 - accuracy: 1.

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 169s 2ms/sample - loss: 2.4797e-06 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 148s 2ms/sample - loss: 8.2844e-06 - accuracy: 1.0000
Epoch 3/10
69687/69687 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 147s 2ms/sample - loss: 1.2576e-06 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 115s 2ms/sample - loss: 5.7404e-06 - accuracy: 1.0000
Epoch 3/10
69687/69687 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 120s 2ms/sample - loss: 1.0503e-06 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 111s 2ms/sample - loss: 9.0240e-07 - accuracy: 1.0000
Epoch 3/10
69687/69687 [================

accuracy_score 0.9999973093977356
recall_score 0.9998236020462162
precision_score 1.0
f1_score 0.9999117932433624
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 113s 2ms/sample - loss: 1.2235e-05 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 103s 1ms/sample - loss: 1.1177e-05 - accuracy: 1.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9998547074777218, 0.9999515691592405, 0.999986546988678, 0.9999946187954711, 1.0, 1.0, 1.0, 1.0, 0.9999973093977356, 1.0]

K-fold cross validation Recall: [0.9956450734893849, 0.9987111029276377, 0.9998169838945827, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9998236020462162, 1.0]

K-fold cross validation Precision: [0.9945622620989668, 0.9979760809567617, 0.9992683372965063, 0.9996438112199466, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9951033732317738, 0.9983434566537824, 0.9995425853078401, 0.9998218738867118, 1.0, 1.0, 1.0, 1.0, 0.9999117932433624, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999784751818848
10-Fold Cross_validation. Recall : 0.9993996762357822
10-Fold Cross_validation. Precision : 0.999145049157218
10-Fold Cross_validation. F1-Score : 0.999272308232347


In [13]:
model1.save('gcc6_bin_core_s48_h72_o1.h5')
print('save 완료')

save 완료
